In [ ]:
import csv
import ndjson
import os
import sys

In [ ]:
base_path = os.getcwd().split('sliver-maestro')[0]
root_path = os.path.join(base_path, "sliver-maestro")
print("root_path: {}".format(root_path))

In [ ]:
sys.path.insert(1, root_path)
from src.utils.model_utils import *
from src.draw_model import *
from src.generate_images import *

In [ ]:
categories_file = os.path.join(root_path, "src", "data", "categories.csv")

In [ ]:
with open(categories_file, newline='') as f:
    reader = csv.reader(f)
    categories = {rows[0]:rows[0] for rows in reader}

#### Select a category to download images from 

In [ ]:
for key, value in categories.items():
    #print(key)
    pass

In [ ]:
print("Selected category from the list above:")
#category=input()
category='cat'
# Checking to see if the entered category is in the categories dictionary
category=categories[category]
print(category)

In [ ]:
print("Creating directories to save images, weights and the model ...")
src_list, dst_list, paths_dict = create_folders(root_path, category)

In [ ]:
print("Select a download method ...")
print("1. Use Google Cloud credentials stored in a json file")
print("2. Use gsutil in the notebook")
print("3. Use gsutil in the terminal")
option = input()

In [ ]:
if option == "1":
    download_data(src_list, dst_list)
elif option == "2":
    for (src_file, dst_file) in zip(src_list, dst_list): 
        !gsutil -m cp {'gs://quickdraw_dataset/' + src_file} {dst_file}
elif option == "3":
    print('Type the following in the terminal to download the files manually:')
    for (src_file, dst_file) in zip(src_list, dst_list):
        print("gsutil -m cp " + "gs://quickdraw_dataset/" + "{}".format(src_file) + " " + "{}".format(dst_file))

In [ ]:
model_path = os.path.join(paths_dict['input_path'], category, category)

In [ ]:
npy_data = np.load(paths_dict['base_input'] + '.npy', allow_pickle=True)

In [ ]:
# load from file-like objects
with open(paths_dict['base_raw'] + '.ndjson') as f:
    ndjson_data = ndjson.load(f)

In [ ]:
def plot_bitmap():
    print("Bitmap image:")
    plt.imshow(npy_data[int(index)].reshape(28,28))

In [ ]:
def plot_raw():
    print("Raw image:")
    i = 0
    for stroke in ndjson_data[int(index)]['drawing']:
        stroke = np.array(stroke)
        plt.plot(stroke[0], stroke[1]*-1)
        i += 1
    print('There are', i, 'number of strokes in this raw image file!')

In [ ]:
print('There are ', len(npy_data)+1, ' images in selected category', categories[category])
print("Select an index value from 0 to", len(npy_data), "to display and compare bitmap and raw images:")
#index=input()
index='0'
plot_bitmap()
plt.show()
plot_raw()

In [ ]:
# TRAIN TEST SPLIT
filepath = paths_dict['input_path']
x_train, x_test, y_train, y_test = split_data(filepath, categories[category], shuffle=False)

In [ ]:
DRAW(model_path, category)

In [ ]:
# TRAINING AND TESTING

#train_data = Dataset(x_train)
#test_data = Dataset(x_test)



model = DRAW(category)
print("training...")
model.start(phase='train') 



## Generating Images

In [ ]:
test_model = Test(category)

In [ ]:
img_loc = {'startr': 0, 'endr': 30, 'startc': 0, 'endc': 30}

print("reconstructing images...")
test_model.xrecons_grid(img_loc)